# Couple particle tracks with background tracer fields to get evolution signatures

In [103]:
import numpy as np
from datetime import timedelta as delta
from datetime import datetime
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
from parcels import FieldSet, Variable, ParticleSet, JITParticle, AdvectionRK4_3D, ErrorCode
import glob

%matplotlib inline

#### Parameters

In [31]:
year = 2002

land_color = '#a9a7a2'

#### Files

In [161]:
# Mesh mask
mesh_mask = '/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc'

# ANHA12 NEMO grid U,V,W files
folder_ANHA12      = '/data/brogalla/ANHA12/'
ANHA12_gridU_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridU.nc'))[0:20]
ANHA12_gridV_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_y{year}m??d??_gridV.nc'))[0:20]
ANHA12_gridW_files = sorted(glob.glob(f'{folder_ANHA12}ANHA12-EXH006_5d_gridW_y{year}m??d??.nc'))[0:20]

# ANHA12 NEMO Pb model files
folder_Pb = f'/data/brogalla/run_storage/Pb-new-ini2-20220922/'
files_Pb  = sorted(glob.glob(f'{folder_Pb}ANHA12_EXH006_5d_{year}0101_{year}1231_ptrc_T_{year}*'))[0:20]

# Create timestamps from file dates:
time_stamps = []
for n, file in enumerate(ANHA12_gridU_files):
    dateU  = datetime.strptime(file[36:47],'y%Ym%md%d')
    dateV  = datetime.strptime(ANHA12_gridV_files[n][36:47],'y%Ym%md%d')
    dateW  = datetime.strptime(ANHA12_gridW_files[n][45:56],'y%Ym%md%d')
    datePb = datetime.strptime(files_Pb[n][99:107],'%Y%m%d')
    
    if (dateU != dateV) | (dateU != dateW) | (dateU != datePb):
        print('Dates not equal!')
    
    # double-check that the date is the same for U, V, and W files
    time_stamps.append(np.datetime64(f'{dateU.year:02}-{dateU.month:02}-{dateU.day:02}'))

timestamps = np.expand_dims(np.array(time_stamps), axis=1)

#### Create fieldset

In [164]:
filenames = {'U': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridU_files},
             'V': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridV_files},
             'W': {'lon': mesh_mask   , 'lat': mesh_mask  , 'depth': ANHA12_gridW_files[0], 'data': ANHA12_gridW_files},
             'Pb': {'lon': files_Pb[0], 'lat': files_Pb[0], 'depth': files_Pb[0], 'data': files_Pb}}

dimensions = {'U': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'V': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'W': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'depthw'},
              'Pb': {'lon': 'nav_lon', 'lat': 'nav_lat', 'depth': 'deptht'}}

variables = {'U': 'vozocrtx',
             'V': 'vomecrty',
             'W': 'vovecrtz',
             'Pb': 'dissolpb'}

# needed to provide time as timestamps in order to avoid time origin type issue
fieldset  = FieldSet.from_nemo(filenames, variables, dimensions, timestamps=timestamps, allow_time_extrapolation=True)

In [165]:
def DeleteParticle(particle, fieldset, time):
    print("Particle [%d] lost !! (%g %g %g %g)" % (particle.id, particle.lon, particle.lat, particle.depth, particle.time))
    particle.delete()

In [166]:
# define new particle class for Pb
class PbParticle(JITParticle): 
    dissolvedPb = Variable('dissolpb', initial=fieldset.Pb)  # Variable 'dissolpb' initialised by sampling the dissolved Pb field

In [167]:
release_depth = 1
release_lon   = -97.45 # east
release_lat   = 74.75  # north
release_time  = 0 # edit!!!

pset = ParticleSet(fieldset=fieldset, pclass=PbParticle, lon=release_lon, lat=release_lat, time=release_time, depth=release_depth)

In [168]:
def SamplePb(particle, fieldset, time):
         particle.dissolpb = fieldset.Pb[time, particle.depth, particle.lat, particle.lon]
        
sample_kernel = pset.Kernel(SamplePb)   

In [81]:
# start_time = datetime.datetime(2015,12,31)
# end_time = timemax*6*3600 # in seconds
# print(start_time - datetime.timedelta(seconds=end_time))

In [ ]:
output_file = pset.ParticleFile(name="test1.nc", outputdt=delta(hours=12))
pset.execute(AdvectionRK4_3D + sample_kernel, runtime=delta(hours=24*30), dt=delta(hours=1),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
             output_file=output_file)
output_file.export()
# dt      --- the timestep of the kernel
# runtime --- the total length of the run

INFO: Compiled PbParticleAdvectionRK4_3DSamplePb ==> /tmp/parcels-2672/5ba55d931cff40bc1b40a6b9658aa1eb_0.so
INFO: Temporary output files are stored in out-GHSEKESS.
INFO: You can use "parcels_convert_npydir_to_netcdf out-GHSEKESS" to convert these to a NetCDF file during the run.
 81% (2116800.0 of 2592000.0) |########  | Elapsed Time: 0:04:19 ETA:   0:00:22

In [93]:
def load_tracks(filename):
    file  = xr.open_dataset(filename)
    traj  = file['trajectory'].values # dimensions: number of particles, tracks
    time  = file['time'].values       # units: seconds
    lat   = file['lat'].values        # degrees North
    lon   = file['lon'].values        # degrees East
    depth = file['z'].values        # degrees East
    dPb   = file['dissolpb'].values        # degrees East

    return traj, time, lon, lat, depth, dPb

In [94]:
traj_t1, time_t1, lon_t1, lat_t1, depth_t1, dPb_t1 = load_tracks( \
                                                '/ocean/brogalla/GEOTRACES/analysis-brogalla/particle_tracking/test.nc')

In [95]:
dPb_t1[0,:]

array([3.0014515e-12, 3.0027317e-12, 3.0041282e-12, 3.0055249e-12,
       3.0069213e-12, 3.0083178e-12, 3.0097144e-12, 3.0111109e-12,
       3.0125076e-12, 3.0139040e-12, 3.0153007e-12, 3.0166971e-12,
       3.0180936e-12, 3.0194903e-12, 3.0208867e-12, 3.0222834e-12,
       3.0157109e-12, 3.0167240e-12, 3.0177369e-12, 3.0187500e-12,
       3.0197630e-12, 3.0207759e-12, 3.0217890e-12, 3.0228021e-12,
       3.0238149e-12, 3.0248280e-12, 3.0258411e-12, 3.0268542e-12,
       3.0278670e-12, 3.0288801e-12, 3.0298932e-12], dtype=float32)

#### Visualize

In [ ]:
import pickle
from mpl_toolkits.basemap import Basemap

In [ ]:
fig, ax1, proj1 = pickle.load(open('/ocean/brogalla/GEOTRACES/pickles/pb-surface.pickle','rb'))

x,y = proj1(lon_t1, lat_t1)

particle = 0
x_plot = x[particle,:][x[particle,:] < 1e10]
y_plot = y[particle,:][y[particle,:] < 1e10]
    
ax1.plot(x_plot, y_plot,'-', linewidth=0.7, color=dPb_t1[particle,:])
ax1.plot(x_plot[-1], y_plot[-1],'.w', markersize=2, markeredgecolor='k')
        
# fig.colorbar(c, shrink=0.5, label='Time [hours]')